# Костыль для исправления файла, честно позаимствован

In [1]:
import re
from numpy import nan
def fix_csv(csv_name):
    csv = open(csv_name, 'r', encoding="UTF-8")
    newcsv = open(csv_name[:-4] + "_R.csv", 'w', encoding="UTF-8")
    names = csv.readline().strip()
    newcsv.write(names.replace(' ', ',') + "\n")
    for line in csv.readlines():
        check = re.search(r'"(.*)"', line)
        if check:
            for match in check.groups():
                line = re.sub(match, match.replace(' ', '_'), line)
        splited_list = line.split(' ')[1:]
        if '' in splited_list:
           splited_list[splited_list.index('')] = str(nan)
        for i in range(len(splited_list)):
            splited_list[i] = splited_list[i].replace('_', ' ')
        newcsv.write(','.join(splited_list))
    csv.close()
    newcsv.close()

# Исправление сломанных файлов

In [2]:
fix_csv('cinema_sessions.csv')
fix_csv('titanic_with_labels.csv')

# Основная часть

In [3]:
import pandas as pd

def select(item, group):
    for group in group:
        if item < group[0]:
            return group[1]

def selectdrink(drink, group):
    for group in group:
        if drink in group[0]:
            return group[1]

df_1 = pd.read_csv('cinema_sessions_R.csv', encoding="UTF-8")
df = pd.read_csv('titanic_with_labels_R.csv', encoding="UTF-8")
df['sex'] = df["sex"].replace(["F", "ж", "Ж", "f"], 1).replace(["M", "m", "М", "м"], 0)
df = df[df['sex'].apply(str).str.isdigit()]
litersDrunkNormalized = df['liters_drunk'][(df['liters_drunk']>= 0) & (df['liters_drunk'] <= 30)]
df.loc[(df['liters_drunk'] <= 0) | (df['liters_drunk'] >= 30), 'liters_drunk'] = litersDrunkNormalized.mean()
df['age'] = df['age'].apply(select, group=[(18, "Child"), (50, "Adult"), (99, "Elderly")] )
df['drink'] = df['drink'].apply(selectdrink, group=[(['Beerbeer', 'Bugbeer', 'Strong beer'], 1),(['Cola', 'Fanta', 'Water'], 0)])
df['row_number'] = df['row_number'].fillna(df['row_number'].max())
df = pd.merge(df, df_1, on="check_number").rename(columns={"session_start" : "session_type"})
df['session_type'] = df['session_type'].apply(select, group=[("12:00:00.000", "Morning"),("17:00:00.000", "Day"),("24:00:00.000", "Evening")])
df

,sex,row_number,liters_drunk,age,drink,check_number,label,session_type
0,1,79.0,2.000000,Elderly,0,3116.0,0.0,Evening
1,0,23.0,2.476779,Child,0,8118.0,0.0,Day
2,0,92.0,2.476779,Adult,0,2264.0,1.0,Evening
3,1,52.0,5.000000,Elderly,1,7640.0,1.0,Evening
4,1,57.0,2.000000,Adult,1,8764.0,0.0,Day
...,...,...,...,...,...,...,...,...
6662,1,100.0,3.000000,Child,0,1022.0,1.0,Day
6663,0,15.0,3.000000,Elderly,1,6510.0,0.0,Evening
6664,1,11.0,3.000000,Elderly,1,9510.0,1.0,Evening
6665,0,96.0,5.000000,Elderly,1,9554.0,0.0,Day
